# Hierarchical Bisecting

In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("hierarchical").getOrCreate()

24/04/03 16:43:44 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface wlo1)
24/04/03 16:43:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/03 16:43:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator

In [3]:
iris = spark.read.csv("../0_data/iris.csv", header=True, inferSchema=True, sep=",")
print(iris.count())
iris.show(5)

150
+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows



In [4]:
vector_assembler = VectorAssembler(inputCols=["sepallength", "sepalwidth", "petallength", "petalwidth"], outputCol="features")
iris_assembled = vector_assembler.transform(iris)
iris_assembled.show(5)

NameError: name 'VectorAssembler' is not defined

In [ ]:
bisec_kmeans = BisectingKMeans(featuresCol="features", k=3, predictionCol="prediction")
model = bisec_kmeans.fit(iris_assembled)
print(model.summary.clusterSizes)

[53, 59, 38]


In [ ]:
clusters = model.transform(iris_assembled)
clusters.show(5)

+-----------+----------+-----------+----------+-----------+-----------------+---------+----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|         features|dependant|prediction|
+-----------+----------+-----------+----------+-----------+-----------------+---------+----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|        0|         0|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|        0|         0|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|        0|         0|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|        0|         0|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|        0|         0|
+-----------+----------+-----------+----------+-----------+-----------------+---------+----------+
only showing top 5 rows



In [ ]:
metrics = ClusteringEvaluator(predictionCol="prediction")
sillouette = metrics.evaluate(clusters)
print(f"Sillouete: {sillouette}")

Sillouete: 0.7231544457999555
